In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'
import numpy as np
import pandas as pd
import skimage.io
import skimage.data
from skimage.color import rgb2gray
from PyTorchNMTF.models import NMF

In [ ]:
%pylab inline

## Data

In [ ]:
im = rgb2gray(skimage.data.chelsea())*255.
im = im.astype('float32')

## Test Image Reconstruction

In [ ]:
device = 'cuda'

In [ ]:
n_iter = 2500
k = 256

In [ ]:
nmf_mu = NMF(im,n_iter=n_iter,k=k,solver='mu',verbose=True,loss='kl')
nmf_mu = nmf_mu.to(device)
nmf_mu.fit()
im_recon_mu = nmf_mu.inverse_trainsform().cpu().numpy()

In [ ]:
nmf_autograd = NMF(im,n_iter=n_iter,k=k,solver='autograd',verbose=True,loss='kl')
nmf_autograd = nmf_autograd.to(device)
nmf_autograd.fit()
im_recon_autograd = nmf_autograd.inverse_trainsform().cpu().numpy()

In [ ]:
figure(figsize=(12,12),facecolor='white')
plt.subplot(221)
imshow(im,cmap='gray')
title('original image')
plt.subplot(222)
imshow(im_recon_mu,cmap='gray')
title('mu solver, mse %.4f' % nmf_mu.mse() )
plt.subplot(223)
opt_name = nmf_autograd.opt.__class__.__name__
title('autograd (%s) solver, mse %.4f' % (opt_name,nmf_autograd.mse()))
imshow(im_recon_autograd,cmap='gray')

In [ ]:
assert nmf_autograd.is_nonneg() and nmf_mu.is_nonneg()